# Numerical data in `numpy`

[`numpy`](https://docs.scipy.org/doc/numpy/) is a very powerful library for working with numerical data in Python. It introduces the __Array__ data structure, which can contain multi-dimensional numerical data. The `numpy` library is __not__ part of the Python standard library. However, it comes bundles with [Anaconda](01_anaconda.ipynb) so you should already have it installed. The usual way to import `numpy` is as follows:

    import numpy as np
    
This gives us access to all the `numpy`-functions using the prefix `np`. This is a convention, and you should do the same in your own code.

In [ ]:
import numpy as np
np.set_printoptions(precision=3, linewidth=75, edgeitems=1)
np.__version__

A simple way to instantiate an array is by giving it a list or another iterable.

In [ ]:
data_in_list = [1, 2, 5, 3]
data_as_np = np.array(data_in_list)
data_as_np

## Reading data with `numpy`

The `numpy` library comes with a few functions that can read numerical data from text files. The `np.loadtxt`-function is a fast reader with some basic functionality for skipping header lines, converting values to floats, etc. For more sophisticated files the `np.genfromtxt`-function can be used. It also supports handling missing values, but is slower.

Note that these functions take a filename as input, so that you do not need to use the built-in `open`-function to open a file beforehand. Assume we have a text-file containing data of the following form:

In [ ]:
!cat data/numpy_simple.txt

These data can be loaded with a very simple `np.loadtxt`-command as follows:

In [ ]:
data = np.loadtxt('data/numpy_simple.txt')
data

Arrays in `numpy` have a shape specifying how big the dataset is. In this case we have 6 rows and 8 columns.

In [ ]:
data.shape

## Indexing and vectorization

Similarly to lists and other sequences in Python, `numpy` arrays can be indexed.

In [ ]:
data[0]    # First row (= row 0)

In [ ]:
data[2:5]    # Rows 2, 3 and 4 (3rd, 4th and 5th)

However, with multi-dimensional arrays (in this case 2-dimensional), we can specify each dimension in the index separated by commas.

In [ ]:
data[4, 5]    # Element in row 4, column 5 (5th row, 6th column)

In [ ]:
data[:, :2]    # All rows, first two columns

With `numpy`-arrays most operations are vectorized. That means that we do not need to explicitly loop over the elements.

In [ ]:
data[:, 4] + data[:, 6]    # Add columns 4 and 6 together

In [ ]:
np.exp(data[:, -1])    # Exponentiate the last (-1) column

`numpy` also comes with summary functions like `sum`, `mean`, `std`, `var`, `median`, etc that can operate on a whole array or a given dimension (`axis`) of the data. 

In [ ]:
np.mean(data)    # Calculate the mean of the whole array

In [ ]:
np.mean(data, axis=0)    # Calculate the mean of each column (along the rows, the 0th axis)

## More advanced reading of data

Most datafiles are not as clean as the simple datafile we have been working with above. Let us instead try to load the following file.

In [ ]:
!cat data/numpy_header.txt

A naive use of `np.loadtxt` will fail because `numpy` tries to interpret the header as data.

In [ ]:
data2 = np.loadtxt('data/numpy_header.txt')

Instead we must give the `np.loadtxt` some more information. To get some help about a function and which parameters it takes, you can write a question mark after its name,

    np.loadtxt?
    
or press `<shift>` and `<tab>` inside the paranthesis. Pressing `<shift>` and `<tab>` twice will give even more information.

In this case, we notice that there is an argument called `skiprows` that can be used to ignore the header.

In [ ]:
data2 = np.loadtxt('data/numpy_header.txt', skiprows=13)
np.allclose(data, data2)    # Test if data and data2 contains the same elements (within a tolerance)

Looking more closely at the data, we also notice that there is one datapoint with the value of -999 that probably designates a missing data point. We can convert this to a `nan`-value to handle it properly as we are reading the data. However, to do so we need to use the more sophisticated `np.genfromtxt`-function.

In [ ]:
data3 = np.genfromtxt('data/numpy_header.txt', skip_header=13,
                      missing_values='-999', usemask=True).filled(np.nan)
data3

Actually handling the missing data is a two-step process. First we create a masked array, where the mask denotes which data are missing. This can be seen if we look directly at the output from `np.genfromtxt` (note the single `True`-value in the mask).

In [ ]:
np.genfromtxt('data/numpy_simple.txt', missing_values='-999', usemask=True)

See the [`numpy` docs](https://docs.scipy.org/doc/numpy-dev/user/basics.io.genfromtxt.html) for more information about reading data.

## Back to our real world example

Let us pick up the example we started in the previous lecture. We are able to read in data in a dirty format using the following code

In [ ]:
from collections import OrderedDict

lines = list()
data = OrderedDict()

def strip_field(field):
    key, plus, value = field.partition('+')
    return value.lstrip('0')

with open('data/dirty_data.txt', mode='r') as fid:
    next(fid)   # Throw away header line
    for line in fid:
        fields = [strip_field(f) for f in line.strip().split()]
        lines.append(fields)
        
for line in lines:
    name = line[0]   # First column is the identifier
    values = [float(v) for v in line[1:-1]]   # Convert to float, ignoring the first and last column
    data.setdefault(name, list()).append(values)   # setdefault creates the list if it does not already exist

### Converting list of lists to `numpy.array`s

In order to make these data easier to work with, we'll now convert each list of lists to a `numpy.array`. We do this by passing each list of lists to a new `OrderedDict` as follows:

In [ ]:
data_np = OrderedDict((k, np.array(v)) for k, v in data.items())
data_np

Calculating the mean of the first column of the `CP4`-data is now trivial:

In [ ]:
data_np['CP4'][:, 0].mean()

### The real world again ...

Of course, we are not interested in a simple mean of these data ...

For each identifier we have 10 measurements. Let us focus on the first column. We notice that every second measurement is approximately 200 away from the previous one. This is by design of the experiment, and an effect we wish to remove. Also, the first two measurements are part of the calibration and should not be included in the final mean.

In [ ]:
means = OrderedDict()

for name, values in data_np.items():
    angles = values[2:, 0].copy()  # First column, skip first two measurements
    angles[1::2] += 200            # Add 200 to every second measurement
    angles = np.mod(angles, 400)   # Modulo 400 to keep in interval (0, 400)
    means[name] = angles.mean()
    
means

## Views and its scary consequences ...

One of the great advantages with `numpy` (in addition to making operations on big data sets convenient) is its speed. When doing operations on whole arrays at once, the heavy lifting is done inside C-code giving quite fast calculations.

One important thing to be aware of is that `numpy`-arrays and views of `numpy`-arrays (sliced sub-arrays) are passed around as references. This means that the code is efficient both in terms of speed and memory, but can also bring some surprises.

Note the `.copy()` done in the line `angles = values[2:, 0].copy()` above. This is not necessary, and actually gives us a small speed penalty. However, lets explore what happens without the copy ...

In [ ]:
values = data_np['CP5']
values

In [ ]:
angles = values[2:, 0]
angles

So far, so good ... Now we'll add 200 to every second element of `angles` ...

In [ ]:
angles[1::2] += 200
angles

In this case (without the `.copy()`) `angles` is a view to the data in `values`. That means that it references the exact same data. So now that we have added 200 to `angles` we have done the same to `values`:

In [ ]:
values

And values was a reference to the data inside the `data_np`-dictionary. So those data are also changed (they are the __same__ data!):

In [ ]:
data_np

In our case this would not have had any immediate consequences. We are after all not using the original data again. However, if we were to run the calculations again, or do some new calculations on the original data we would have gotten weird head-scratching bugs to deal with ...

Note that this is a __feature__ of Python and `numpy` (so no reason to rush over to [bugs.python.org/](https://bugs.python.org/)). But it is important to be aware of, and to be conscious about changing data referenced by several names.